# Add unique identifiers to your tables

Generate UUIDs for automatic row identification.


## Problem

You need unique identifiers for rows in your data pipeline. Maybe you're building an API that returns specific records, tracking processing status across systems, or joining data from multiple sources.

| Use case | Why UUIDs help |
|----------|----------------|
| API responses | Return stable IDs clients can reference |
| Distributed systems | Generate IDs without coordination |
| Data lineage | Track records across pipeline stages |


## Solution

**What's in this recipe:**
- Create tables with auto-generated UUID primary keys
- Add UUID columns to existing tables
- Generate UUIDs with `uuid4()`

You use `uuid4()` to generate UUIDs for each row. Define it in the schema with `{'value': uuid4()}` syntax, or add it to existing tables with `add_computed_column()`.


### Setup


In [1]:
# For development: use local pixeltable
import sys
sys.path.insert(0, '/Users/pierre/pixeltable')

# For production: uncomment the line below
# %pip install -qU pixeltable


In [ ]:
import pixeltable as pxt
from pixeltable.functions.uuid import uuid4


In [3]:
# Create a fresh directory
pxt.drop_dir('uuid_demo', force=True)
pxt.create_dir('uuid_demo')


Connected to Pixeltable database at: postgresql+psycopg://postgres:@/pixeltable?host=/Users/pjlb/.pixeltable/pgdata
Created directory 'uuid_demo'.


### Create a table with a UUID primary key

Use `{'value': uuid4()}` in your schema to create a column that auto-generates UUIDs:


In [ ]:
# Create table with auto-generated UUID primary key
products = pxt.create_table(
    'uuid_demo.products',
    {
        'id': {'value': uuid4()},  # Auto-generates UUID for each row
        'name': pxt.String,
        'price': pxt.Float
    },
    primary_key=['id']
)


Created table 'products'.


In [5]:
# Insert data - no need to provide 'id', it's auto-generated
products.insert([
    {'name': 'Laptop', 'price': 999.99},
    {'name': 'Mouse', 'price': 29.99},
    {'name': 'Keyboard', 'price': 79.99}
])


Inserting rows into `products`: 3 rows [00:00, 215.15 rows/s]
Inserted 3 rows with 0 errors.


3 rows inserted, 6 values computed.

In [6]:
# View the data - each row has a unique UUID
products.collect()


id,name,price
dda9047a-150c-4230-937e-3370d9d499b0,Laptop,999.99
8d06596e-c8a7-4c89-b018-989c7f1623ad,Mouse,29.99
12fa77c1-b227-43d9-a9d3-4e915b36fb97,Keyboard,79.99


### Add a UUID column to an existing table

You can add a UUID column to a table that already exists using `add_computed_column()`:


In [ ]:
# Create a table without a UUID column
orders = pxt.create_table(
    'uuid_demo.orders',
    {
        'customer': pxt.String,
        'amount': pxt.Float
    }
)


Created table 'orders'.


In [8]:
# Insert some data
orders.insert([
    {'customer': 'Alice', 'amount': 150.00},
    {'customer': 'Bob', 'amount': 75.50}
])


Inserting rows into `orders`: 2 rows [00:00, 1176.36 rows/s]
Inserted 2 rows with 0 errors.


2 rows inserted, 2 values computed.

In [ ]:
# Add a UUID column to existing table
orders.add_computed_column(order_id=uuid4())


Error: The argument to add_column() must be a type; did you intend to use add_computed_column() instead?

In [ ]:
# View orders with their UUID column
orders.collect()


### Use UUIDs in computed columns

You can also use `uuid4()` directly in computed columns for any table:


In [ ]:
# Create a table with a computed UUID column
events = pxt.create_table(
    'uuid_demo.events',
    {
        'event_type': pxt.String,
        'data': pxt.Json
    }
)


In [ ]:
# Add a computed column that generates UUIDs
events.add_computed_column(event_id=uuid4())


In [ ]:
# Insert events
events.insert([
    {'event_type': 'click', 'data': {'button': 'submit'}},
    {'event_type': 'view', 'data': {'page': '/home'}}
])


In [ ]:
# View events with their generated UUIDs
events.collect()


## Explanation

**Two ways to add UUIDs:**

| Approach | Use case |
|----------|----------|
| `{'value': uuid4()}` in schema | Auto-generated UUID at table creation |
| `add_computed_column(col=uuid4())` | Add UUID column to existing table |

Both use `uuid4()` which generates UUIDv4 (random) identifiers:
- 128-bit values
- Formatted as 32 hex digits with hyphens: `550e8400-e29b-41d4-a716-446655440000`
- Virtually guaranteed unique (collision probability is negligible)


## See also

- [Tables and operations](https://docs.pixeltable.com/datastore/tables-and-operations)
- [Computed columns](https://docs.pixeltable.com/datastore/computed-columns)
